In [ ]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import networkx as nx
from pathlib import Path
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', 500)

from sklearn.preprocessing import MinMaxScaler
import sys
sys.path.append('/Users/david/Dropbox/PhD/Scripts/Spatial analyses')
import pyspace
import libpysal as lps
from scipy.spatial import cKDTree
from libpysal.weights.distance import get_points_array
from esda import fdr
from importlib import reload
pd.set_option('display.max_rows', 500)
reload(pyspace)
import seaborn as sns
from esda.moran import Moran
# sns.set_theme(font = 'Helvetica')
%matplotlib inline
from numba import NumbaDeprecationWarning
import warnings
import os
# Suppress NumbaDeprecationWarning
warnings.filterwarnings("ignore", category=NumbaDeprecationWarning)
from utils import read_data, plot_covariate_distributions, plot_match, compare_balance, sizeof_fmt, optimize_memory_df, plot_categorical_proportional_diff, compute_mean_differences_and_proportions, love_plot, sensitivity_analysis_k_neighbors

In [ ]:
data_folder  = Path('../data/')
results_folder = Path('../output/')
model_folder = results_folder/'Models'

if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [ ]:
df_treated_filtered = pd.read_parquet(data_folder/'processed/df_treated_filtered_nominors.parquet.gzip')

In [ ]:
df_treated_filtered.uuid.nunique()

In [ ]:
df_treated_filtered.groupby(['NOANNEE','CDPHYSSEXE'])['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack(level=1).div(1000).round(1)

In [ ]:
test = pd.concat([df_treated_filtered.groupby(['NOANNEE','CDPHYSSEXE'])['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack().div(1000).round(1),
df_treated_filtered.groupby(['NOANNEE','age_group'],observed=True)['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack().div(1000).round(1),
df_treated_filtered.groupby(['NOANNEE','ssep3_q'],observed=True)['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack().div(1000).round(1),
df_treated_filtered.groupby(['NOANNEE','Language'],observed=True)['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack().div(1000).round(1),
df_treated_filtered.groupby(['NOANNEE','CAREMODEL'],observed=True)['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack().div(1000).round(1),
df_treated_filtered.groupby(['NOANNEE','MTFRANCHISECOUV'],observed=True)['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack().div(1000).round(1)], axis=1).T.reset_index()
test.columns = ['Statistic', 'Categories', '2017', '2018', '2019', '2020', '2021']

# Pivot the DataFrame to make each row represent one year and one sex
df_pivot = test.pivot(index='Categories', columns='Statistic')

for year in ['2017', '2018', '2019', '2020', '2021']:
    df_pivot[year] = df_pivot[(year, 'mean')].astype(str) + " (" + df_pivot[(year, 'std')].astype(str) + ")"
df_pivot.columns = df_pivot.columns.droplevel(1)  # Drops the upper level, leaving only the years
df_final = df_pivot.reset_index()
df_final = df_final.loc[:,~df_final.columns.duplicated()].copy()
df_final.to_clipboard()

In [ ]:
def percentile_25(x):
    return np.percentile(x, 25)

def percentile_75(x):
    return np.percentile(x, 75)

def median(x):
    return np.median(x)

In [ ]:
test = pd.concat([df_treated_filtered.groupby(['NOANNEE','CDPHYSSEXE'])['PRESTATIONS_BRUTES_LCA'].agg({"mean","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','age_group'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"mean","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','ssep3_q'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"mean","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','Language'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"mean","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','CAREMODEL'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"mean","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','MTFRANCHISECOUV'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"mean","std"}).unstack().round(1)], axis=1).T.reset_index()
test.columns = ['Statistic', 'Categories', '2017', '2018', '2019', '2020', '2021']

# Pivot the DataFrame to make each row represent one year and one sex
df_pivot = test.pivot(index='Categories', columns='Statistic')

for year in ['2017', '2018', '2019', '2020', '2021']:
    df_pivot[year] = df_pivot[(year, 'mean')].astype(str) + " (" + df_pivot[(year, 'std')].astype(str) + ")"
df_pivot.columns = df_pivot.columns.droplevel(1)  # Drops the upper level, leaving only the years
df_final = df_pivot.reset_index()
df_final = df_final.loc[:,~df_final.columns.duplicated()].copy()
df_final.to_clipboard()

In [ ]:
pd.concat([df_treated_filtered.groupby(['NOANNEE','CDPHYSSEXE'])['PRESTATIONS_BRUTES_LCA'].agg({"median","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','age_group'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"median","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','ssep3_q'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"median","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','CDLANGUE'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"median","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','CAREMODEL'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"median","std"}).unstack().round(1)], axis=1).T.reset_index()

In [ ]:
df_uuid = df_treated_filtered[['uuid']]
df_uuid = df_uuid.drop_duplicates().reset_index(drop=True)

### Figure S2A - Trends in healthcare expenditures (2017-2021)

In [ ]:
yearly_data_AOS = df_treated_filtered.groupby('NOANNEE')['PRESTATIONS_BRUTES_AOS'].sum().div(1000000).reset_index()
yearly_data_LCA = df_treated_filtered.groupby('NOANNEE')['PRESTATIONS_BRUTES_LCA'].sum().div(1000000).reset_index()
yearly_data_CAM = df_treated_filtered.groupby('NOANNEE')['PRESTATIONS_BRUTES_CAM'].sum().div(1000000).reset_index()

# Create a long format DataFrame for easier plotting with seaborn
combined_data = pd.concat([
    yearly_data_AOS.rename(columns={'PRESTATIONS_BRUTES_AOS': 'Values'}).assign(Type='CM - MHI'),
    yearly_data_CAM.rename(columns={'PRESTATIONS_BRUTES_CAM': 'Values'}).assign(Type='CAM - MHI'),
    yearly_data_LCA.rename(columns={'PRESTATIONS_BRUTES_LCA': 'Values'}).assign(Type='CAM - SI')
])

# Adjusted for FacetGrid with custom x-ticks
def plot_facet_grid(data, x, y, facet_col, x_label, y_label):
    g = sns.FacetGrid(data, row=facet_col, sharey=False, height=4, aspect=2)
    g.map_dataframe(sns.lineplot, x=x, y=y, marker='o')
    g.set_axis_labels(x_label, y_label)

    # Set x-ticks for each facet
    for ax in g.axes.flatten():
        ax.set_xticks(data[x].unique())  # Assuming x-values are year numbers
        ax.set_xticklabels(data[x].unique(), rotation=45)
        ax.grid(True)

# Combine and rename data as per previous example for combined_data
plot_facet_grid(
    combined_data,
    'NOANNEE', 'Values', 'Type',
    'Year', 'Total annual expenditures (millions CHF)'
)
plt.savefig(results_folder/'Fig S2A - Evolution_costs.png', dpi = 300, bbox_inches='tight')

### Figure S2B - Trends in healthcare expenditures (2017-2021)

In [ ]:
yearly_data_AOS = df_treated_filtered.groupby('NOANNEE')['PRESTATIONS_BRUTES_AOS'].sum().div(1000000).reset_index()
yearly_data_LCA = df_treated_filtered.groupby('NOANNEE')['PRESTATIONS_BRUTES_LCA'].sum().div(1000000).reset_index()
yearly_data_CAM = df_treated_filtered.groupby('NOANNEE')['PRESTATIONS_BRUTES_CAM'].sum().div(1000000).reset_index()

# Normalize data

yearly_data_AOS['PRESTATIONS_BRUTES_AOS'] = yearly_data_AOS['PRESTATIONS_BRUTES_AOS'] / yearly_data_AOS['PRESTATIONS_BRUTES_AOS'].iloc[0] * 100
yearly_data_LCA['PRESTATIONS_BRUTES_LCA'] = yearly_data_LCA['PRESTATIONS_BRUTES_LCA'] / yearly_data_LCA['PRESTATIONS_BRUTES_LCA'].iloc[0] * 100
yearly_data_CAM['PRESTATIONS_BRUTES_CAM'] = yearly_data_CAM['PRESTATIONS_BRUTES_CAM'] / yearly_data_CAM['PRESTATIONS_BRUTES_CAM'].iloc[0] * 100

# Plot normalized data
fig, ax = plt.subplots(figsize=(8, 4))
sns.lineplot(data=yearly_data_AOS, x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', marker='o', label='CM - MHI', ax=ax)
sns.lineplot(data=yearly_data_LCA, x='NOANNEE', y='PRESTATIONS_BRUTES_LCA', marker='o', label='CAM - SI', ax=ax)
sns.lineplot(data=yearly_data_CAM, x='NOANNEE', y='PRESTATIONS_BRUTES_CAM', marker='o', label='CAM - MHI', ax=ax)
ax.set_xticks(yearly_data_CAM['NOANNEE'])  # Assuming x-values are year numbers
ax.set_xticklabels(yearly_data_CAM['NOANNEE'].astype('string'), rotation=45)
ax.grid(True)

plt.xlabel('Year')
plt.ylabel('Evolution of annual expenditures (%)')  # Update label to indicate these are percentages
plt.grid(True)
plt.legend(title='Prestation type')
# plt.show()
plt.savefig(results_folder/'Fig S2B - Evolution_costs_perc_baseline.png', dpi = 300, bbox_inches='tight')

### Cost distribution after transformation

#### AOS

In [ ]:
df_treated_filtered['n_atc'].plot.hist(bins = 100)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16, 4))

g = sns.kdeplot(data=df_treated_filtered, x="PRESTATIONS_BRUTES_AOS", ax = axs[0])
g.set_xlabel('Yearly CM claims amount (MHI) (CHF)')
g = sns.kdeplot(data=df_treated_filtered, x="ihs_cost_aos", ax = axs[1])
g.set_xlabel('IHS transformed yearly CM claims amount (MHI) (CHF)')
g = sns.kdeplot(data=df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_AOS > df_treated_filtered.MTFRANCHISECOUV], x="ihs_cost_aos", ax = axs[2])
g.set_xlabel('IHS transformed yearly CM claims amount (MHI) (CHF)')
plt.savefig(results_folder/'IHS_transformation_procedure_AOS.png', dpi = 300, bbox_inches='tight')

#### LCA

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16, 4))

g = sns.kdeplot(data=df_treated_filtered, x="PRESTATIONS_BRUTES_LCA", ax = axs[0])
g.set_xlabel('Yearly CAM claims amount (SI) (CHF)')
g = sns.kdeplot(data=df_treated_filtered, x="ihs_cost_lca", ax = axs[1])
g.set_xlabel('IHS transformed yearly CAM claims amount (SI) (CHF)')
g = sns.kdeplot(data=df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_LCA > df_treated_filtered.MTFRANCHISECOUV], x="ihs_cost_lca", ax = axs[2])
g.set_xlabel('IHS transformed yearly CAM claims amount (SI) (CHF)')
plt.savefig(results_folder/'IHS_transformation_procedure_LCA.png', dpi = 300, bbox_inches='tight')

## Création de sous-groupes

- Répondre à des questions spécifiques
- Augmenter l'homogénéité de la population étudiée

In [ ]:
# Healthy = People with PCG flags = 0 in any year
uuid_healthy = df_treated_filtered[df_treated_filtered.n_flags == 0].uuid.unique()
df_healthy = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_healthy)]

# Healthy = People with AOS = 0 at baseline
uuid_healthy_bs = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.n_flags == 0)].uuid.unique()
df_healthy_bs = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_healthy_bs)]

# Patients with multimorbidity
uuid_multimorbid = df_treated_filtered[df_treated_filtered.multimorbidity == 1].uuid.unique()
df_multimorbid = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_multimorbid)]

# Patients with multimorbidity at baseline
uuid_multimorbid_bs = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.multimorbidity == 1)].uuid.unique()
df_multimorbid_bs = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_multimorbid_bs)]

# Patients with cancer
uuid_cancer = df_treated_filtered[(df_treated_filtered.Cancer_PCG == 1)].uuid.unique()
df_cancer = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_cancer)]

# Patients with cancer at baseline
uuid_cancer_bs = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.Cancer_PCG == 1)].uuid.unique()
df_cancer_bs = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_cancer_bs)]

# Patients with diabetes
uuid_diab = df_treated_filtered[(df_treated_filtered.Diabetes_PCG == 1)].uuid.unique()
df_diab = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_diab)]

# Patients with diabetes at baseline
uuid_diab_bs = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.Diabetes_PCG == 1)].uuid.unique()
df_diab_bs = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_diab_bs)]

# Patients with pain
uuid_pain = df_treated_filtered[(df_treated_filtered.Pain_PCG == 1)].uuid.unique()
df_pain = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_pain)]

# Patients with pain
uuid_mental = df_treated_filtered[(df_treated_filtered.Mental_PCG == 1)].uuid.unique()
df_mental = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_mental)]


## Total spending by category

In [ ]:
variable_names = pd.DataFrame({"old": ['multimorbidity','ssep3_q','Urbanicity_simple','n_atc','n_flags','NBAGE',"NBAGE_std",'age_group', "ssep3_std",'ssep3', 'region_DE', 'region_FR', 'region_IT','urb_Urbain','urb_Périurbain','Asthma_PCG', 'Cancer_PCG', 'Diabetes_PCG', 'Epilepsy_PCG',
       'Glaucoma_PCG', 'HIV_AIDS_PCG', 'Heart_disease_PCG',
       'Hypertension_related_PCG', 'Immune_PCG', 'Inflammatory_PCG',
       'Mental_PCG', 'Other_PCG', 'Pain_PCG', 'Parkinson_PCG', 'Thyroid_PCG', "SEX_F",'SEX','CDPHYSSEXE','LANG', "cds_std",'cds','LANG_FR','D_MEDIC_B','D_MEDIC_S','D_MEDIC_B_std','D_MEDIC_S_std','DEDUCTIBLE_above_500','E_std','N_std','E_std:N_std','PRESTATIONS_BRUTES_ATC','PRESTATIONS_TOTAL','PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_LCA','PRESTATIONS_BRUTES_CAM','PRESTATIONS_BRUTES_AMBULATOIRE','PRESTATIONS_BRUTES_STATIONNAIRE','PRESTATIONS_ACCIDENT','PRESTATIONS_DISEASE','PRESTATIONS_BIRTH','MTFRANCHISECOUV','mean_pm10','mean_no2','mean_pm25','mean_ndvi','mean_lst','mean_carnight'],
                           "new": ['Multimorbidity','Swiss-SEP','Urbanicity','Number of ATC','Number of PCG flags','Age',"Age", 'Age Group', "SES index",'SES index','German', 'French', 'Italian','Urban','Periurban', 'Asthma', 'Cancer', 'Diabetes', 'Epilepsy', 'Glaucoma', 'HIV/AIDS',
       'Heart disease', 'Hypertension related', 'Immune', 'Inflammatory',
       'Mental', 'Other', 'Pain', 'Parkinson', 'Thyroid', "Sex (Female)",'Sex','Sex','Langage', "CDS",'CDS','French speaker','Access to prim. care med.','Access to spec. med.','Access to prim. care med.','Access to spec. med.','Deductible (>500)','E','N','E:N','Drug-related claims (CHF)','Total claims amount (CHF)','CM claims (MHI) (CHF)','CAM claims (SI) (CHF)','CAM claims (MHI) (CHF)','Ambulatory claims (CHF)','Stationary claims (CHF)','Accident-related claims (CHF)','Disease-related claims (CHF)','Birth-related claims (CHF)','Deductible','PM10','NO2','PM25','NDVI','LST','Nighttime car noise']})
rename_dict = variable_names.set_index('old')['new'].to_dict()

def update_variable_names(summary_table, variable_names, table_type):
    name_mapper = variable_names.set_index('old')['new'].to_dict()
    if table_type == 'summary':
        name_mapper = {f"{key}, mean (SD)": f"{value}, mean (SD)" for key, value in name_mapper.items()}
    elif table_type == 'categorical':
        name_mapper = {f"{key}, n (%)": f"{value}, n (%)" for key, value in name_mapper.items()}
    summary_table = summary_table.rename(index=name_mapper)
    return summary_table

In [ ]:
df_treated_filtered_long = df_treated_filtered.set_index('NOANNEE')[['PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_CAM','PRESTATIONS_BRUTES_LCA']].stack().reset_index()

df_treated_filtered_long.columns = ['NOANNEE','cat_spending','amount']

# sns.barplot(df_treated_filtered_long, x="NOANNEE", y="amount", hue = 'cat_spending')

In [ ]:
# Assuming df_treated_filtered_long is your DataFrame after manipulation
df_treated_filtered_long['cat_spending'] = df_treated_filtered_long['cat_spending'].replace({
    'PRESTATIONS_BRUTES_AOS': 'Conventional medicine (MHI)',
    'PRESTATIONS_BRUTES_CAM': 'Complementary medicine (MHI)',
    'PRESTATIONS_BRUTES_LCA': 'Complementary medicine (SI)'
})

In [ ]:
palette = {
    'Conventional medicine (MHI)': '#4E79A7',
    'Complementary medicine (MHI)': '#59A14F',
    'Complementary medicine (SI)': '#F28E2B'
}

In [ ]:
# Now plot with Seaborn
g = sns.barplot(data=df_treated_filtered_long, x="NOANNEE", y="amount", hue='cat_spending', palette=palette)
# plt.ylim([0,10000])
# g.set_yscale("log")
# the non-logarithmic labels you want
# ticks = [1, 10, 100, 1000, 10000, 100000]
# g.set_yticks(ticks)
# g.set_yticklabels(ticks)
g.set_xlabel('Year')
plt.legend(title='')
# If you want to further customize the legend, you can use plt.legend:
# plt.legend(title='S')
plt.ylabel('Average yearly claims amount (CHF)')
# To ensure the plot displays properly in Jupyter notebooks
plt.savefig(results_folder/'Avg_expenditures.png', dpi = 300, bbox_inches='tight')

In [ ]:
# Now plot with Seaborn
g = sns.barplot(data=df_treated_filtered_long, x="NOANNEE", y="amount", hue='cat_spending', palette=palette)
# plt.ylim([0,10000])
g.set_yscale("log")
# the non-logarithmic labels you want
ticks = [1, 10, 100, 1000, 10000, 100000]
g.set_yticks(ticks)
g.set_yticklabels(ticks)
g.set_xlabel('Year')
plt.legend(title='')
# If you want to further customize the legend, you can use plt.legend:
# plt.legend(title='S')
plt.ylabel('Average yearly claims amount (CHF)')
# To ensure the plot displays properly in Jupyter notebooks
plt.savefig(results_folder/'Avg_expenditures_log.png', dpi = 300, bbox_inches='tight')

In [ ]:
# First subplot
g = sns.lineplot(x='NOANNEE', y='amount', hue='cat_spending', data=df_treated_filtered_long, marker='o')
g.set_yscale("log")
# the non-logarithmic labels you want
ticks = [1, 10, 100, 1000, 10000, 100000]
g.set_yticks(ticks)
g.set_yticklabels(ticks)
g.set_xlabel('Year')
plt.legend(title='')


In [ ]:
df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_CAM > 0].groupby('NOANNEE').uuid.nunique().plot.bar()

In [ ]:
df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_LCA > 0].groupby('NOANNEE').uuid.nunique().plot.bar()

In [ ]:
g = sns.barplot(df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_CAM > 0], x="NOANNEE", y="PRESTATIONS_BRUTES_CAM", color = '#59A14F')
g.set_ylabel('Average yearly claims amount (CHF)')
# g.legend(title='Complementary medicine (LAMal) for users')
g.set_xlabel('Year')
g.set_ylim([0,1500])
plt.savefig(results_folder/'Avg_expenditures_CAM_users.png', dpi = 300, bbox_inches='tight')

In [ ]:
df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_CAM > 0].groupby('year')['PRESTATIONS_BRUTES_CAM'].describe()

In [ ]:
sns.barplot(df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_LCA > 0], x="NOANNEE", y="PRESTATIONS_BRUTES_LCA")

### Number of individuals by cumulated years of treatment

In [ ]:
ttmt_month_gp_uuid = df_treated_filtered.groupby('uuid')['n_month_lca_by_patient'].sum().to_dict()
df_treated_filtered['cumulated_n_month_lca_by_patient'] = df_treated_filtered['uuid'].map(ttmt_month_gp_uuid)
df_uuid['cumulated_n_month_lca_by_patient'] = df_uuid['uuid'].map(ttmt_month_gp_uuid)

In [ ]:
df_treated_filtered.groupby('uuid')['PRESTATIONS_BRUTES_LCA'].sum().plot.hist(bins = 100)

In [ ]:
# df_treated_filtered.groupby('treatment_cumulated_lca_cam').uuid.nunique().plot.bar()
from matplotlib.ticker import FuncFormatter

# First, perform your groupby operation and reset the index to turn it into a DataFrame
df_aggregated = df_treated_filtered.groupby('treatment_cumulated_lca_cam')['uuid'].nunique().reset_index()

# Rename the columns for clarity if desired
df_aggregated.columns = ['Treatment', 'UniqueCount']
# Now, use Seaborn to create a bar plot
def thousands_separator(x, pos):
    return '{:,.0f}'.format(x)  # The '0f' ensures no decimal places

# Assuming df_aggregated is your DataFrame after the groupby and reset_index
sns.barplot(x='Treatment', y='UniqueCount', data=df_aggregated, palette = 'Blues')

# Get the current axis
ax = plt.gca()

# Set the formatter for the y-axis to use your custom formatter
ax.yaxis.set_major_formatter(FuncFormatter(thousands_separator))

# Optional: add labels and title for clarity
plt.xlabel('Number of years in the treatment group (CAM claims amount > 0) \n (MHI and SI combined)')
plt.ylabel('Number of individuals')
# plt.title('Number of individuals by Treatment')
plt.savefig(results_folder/'Number_users_CAM_cumulated_years.png', dpi = 300, bbox_inches='tight')

In [ ]:
df_aggregated['perc_count'] = (df_aggregated['UniqueCount']/df_aggregated['UniqueCount'].sum())*100

In [ ]:
df_treated_filtered.groupby('cumulated_n_month_lca_by_patient').uuid.nunique().plot.bar(figsize = (12,8))

In [ ]:
df_uuid['cumulated_n_month_lca_by_patient'].median()

## What generates spending in AOS?

#### Time-varying covariates (proxies for health status)
- Age
- Sex
- Deductible
- Education, income (Swiss SES index)
- Access to healthcare
    - Captured directly through measured access (existing variables : d_medic_s, d_medic)
    - Captured indirectly through urban contexts (urban, periurban, rural)
- Disease state : CDS, PCGs, HOSPITALISATIONS

#### Time-invariant covariates
- Sociocultural norms:
    - Captured through the canton of residence and spoken langage
- Policies :
    - Captured through the canton of residence 
- Environmental factors:
    - Air quality (NO2, PM10, PM2.5)
    - Vegetation (NDVI)
    - Noise (dB)

#### Time-varying treatment
- Treatment

### Health status -> cost, treatment

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# First subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_AOS", x='n_flags', width=.6, palette="vlag", showfliers=False, ax=axs[0, 0])

# Second subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_DISEASE", x='n_flags', width=.6, palette="vlag", showfliers=False, ax=axs[0, 1])

# Third subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_LCA", x='n_flags', width=.6, palette="vlag", showfliers=False, ax=axs[1, 0])

# Fourth subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_AOS", x='locdrhosp', width=.6, palette="vlag", showfliers=False, ax=axs[1, 1])

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(6, 4))

# First subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_AOS", x='multimorbidity', width=.6, palette="vlag", showfliers=False, ax=axs[0,0])

# Second subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_LCA", x='multimorbidity', width=.6, palette="vlag", showfliers=False, ax=axs[0,1])

# Third subplot
sns.barplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_CAM", x='multimorbidity', width=.6, palette="vlag", ax=axs[1,0])

# Fourth subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_DISEASE", x='multimorbidity', width=.6, palette="vlag", showfliers=False, ax=axs[1,1])


# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
df_treated_filtered.groupby('multimorbidity')['PRESTATIONS_BRUTES_CAM'].mean()

#### Remarques

- Les personnes avec des comorbidités ne sont pas celles qui utilisent plus de LCA
- Les personnes avec des comorbidités dépensent bcp plus en AOS (logique)

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(10, 6))

# First subplot
g = sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_AOS", x='CDPHYSSEXE', width=.6, palette="vlag", showfliers=False, ax=axs[0, 0])
g.set_xlabel('Sex')
g.set_ylabel('Annual CM claims amount (MHI) (CHF)')
# Second subplot
g = sns.boxplot(data=df_treated_filtered, y="cds", x='CDPHYSSEXE', width=.6, palette="vlag", showfliers=False, ax=axs[0, 1])
g.set_xlabel('Sex')
g.set_ylabel('Chronic Disease Score (CHF)')
# Third subplot
g = sns.barplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_CAM", x='CDPHYSSEXE', width=.6, palette="vlag", ax=axs[1, 0])
g.set_xlabel('Sex')
g.set_ylabel('Annual CAM claims amount (MHI) (CHF)')
# Fourth subplot
g = sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_LCA", x='CDPHYSSEXE', width=.6, palette="vlag",showfliers=False, ax=axs[1, 1])
g.set_xlabel('Sex')
g.set_ylabel('Annual CAM claims amount (SI) (CHF)')
# Adjust layout
plt.tight_layout()
plt.savefig(results_folder/'Fourplot_cost_by_sex.png', dpi = 300, bbox_inches='tight')

### Age -> cost, disease state, treatment

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(10, 6))

# First subplot
g = sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_AOS", x='age_group', width=.6, palette="vlag", showfliers=False, ax=axs[0, 0])
g.set_xlabel('Age group')
g.set_ylabel('Annual CM claims amount (MHI) (CHF)')
# Second subplot
g = sns.boxplot(data=df_treated_filtered, y="cds", x='age_group', width=.6, palette="vlag", showfliers=False, ax=axs[0, 1])
g.set_xlabel('Age group')
g.set_ylabel('Chronic Disease Score (CHF)')
# Third subplot
g = sns.barplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_CAM", x='age_group', width=.6, palette="vlag", ax=axs[1, 0])
g.set_xlabel('Age group')
g.set_ylabel('Annual CAM claims amount (MHI) (CHF)')
# Fourth subplot
g = sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_LCA", x='age_group', width=.6, palette="vlag",showfliers=False, ax=axs[1, 1])
g.set_xlabel('Age group')
g.set_ylabel('Annual CAM claims amount (SI) (CHF)')
# Adjust layout
plt.tight_layout()
plt.savefig(results_folder/'Fourplot_cost_by_age.png', dpi = 300, bbox_inches='tight')

## Time -> Cost, Disease, Treament

In [ ]:
palette = sns.color_palette()


dic_labels = {1:'Usage',0:'No usage'}

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# List of dataframes and titles
dataframes = [df_treated_filtered, df_healthy, df_multimorbid, df_cancer]
titles = ['All individuals', 'Individuals without chronic conditions', 'Multimorbid individuals', 'Individuals with cancer']

# Create subplots
for i, (ax, df, title) in enumerate(zip(axs.flat, dataframes, titles)):
    sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cam_only', data=df, marker='o', ax=ax, legend=False)
    ax.set_title(title)
    ax.set_xlabel('Year')
    ax.set_ylabel('Conventional Medicine\nexpenditures (CHF)')
    ax.grid(True)

# Manually create legend handles and labels
legend_handles = [plt.Line2D([0], [0], marker='o', color=palette[i], 
                             label=dic_labels[treatment], markersize=8, linestyle='-') 
                  for i, treatment in enumerate(df_treated_filtered['treatment_cam_only'].unique())]

# Create a single legend with the correct title
legend = fig.legend(handles=legend_handles, 
           title='Complementary/Alternative Medicine (CAM)\nthrough Mandatory Health Insurance (MHI)', 
           loc='lower center', alignment='center', bbox_to_anchor=(0.5, -0.04), ncol=2)
plt.setp(legend.get_title(), ha='center')  # 'ha' stands for horizontal alignment

# Adjust layout
plt.tight_layout()
fig.subplots_adjust(bottom=0.15)  # Make room for the legend

plt.savefig(model_folder/'trends_treatment_cam_only_aos_cost.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# First subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cam_only', data=df_treated_filtered, marker='o', ax=axs[0, 0])
axs[0, 0].set_title('All')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Expenditures CM - MHI  (CHF)')
axs[0, 0].legend(title='CAM - MHI usage', loc='upper left')
axs[0, 0].grid(True)
# Second subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cam_only', data=df_healthy, marker='o', ax=axs[0, 1])
axs[0, 1].set_title('No-PCG individuals')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Expenditures CM - MHI  (CHF)')
axs[0, 1].legend(title='CAM - MHI usage', loc='upper left')
axs[0, 1].grid(True)

# Third subplot
# Create the line plot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cam_only', data=df_multimorbid, marker='o', ax=axs[1, 0])
axs[1, 0].set_title('Multimorbid patients')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Expenditures CM - MHI  (CHF)')
axs[1, 0].legend(title='CAM - MHI usage', loc='upper left')
axs[1, 0].grid(True)

# Fourth subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cam_only', data=df_cancer, marker='o', ax=axs[1, 1])

axs[1, 1].set_title('Patients with cancer')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Expenditures CM - MHI (CHF)')
axs[1, 1].legend(title='CAM - MHI usage', loc='upper left')
axs[1, 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.savefig(model_folder/'trends_treatment_cam_only_aos_cost.png', dpi = 300, bbox_inches = 'tight')


In [ ]:
df_spenders_cam_lca = df_treated_filtered[df_treated_filtered.treatment_cam_only == 1]

In [ ]:
df_treated_filtered[df_treated_filtered.treatment_cumulated == 1].uuid.nunique()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 7))

# First subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_LCA', hue='treatment_cam_only', data=df_treated_filtered, marker='o', ax=axs[0, 0])
axs[0, 0].set_title('All')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Average Spending on Complementary Medicine')
axs[0, 0].legend(title='CAM - MHI usage', loc='upper left')
axs[0, 0].grid(True)
# Second subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_LCA', hue='treatment_cam_only', data=df_healthy, marker='o', ax=axs[0, 1])
axs[0, 1].set_title('No-PCG individuals')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Average Spending on Complementary Medicine')
axs[0, 1].legend(title='CAM - MHI usage', loc='upper left')
axs[0, 1].grid(True)

# Third subplot
# Create the line plot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_LCA', hue='treatment_cam_only', data=df_multimorbid, marker='o', ax=axs[1, 0])
axs[1, 0].set_title('Multimorbid patients')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Average Spending on Complementary Medicine')
axs[1, 0].legend(title='CAM - MHI usage', loc='upper left')
axs[1, 0].grid(True)

# Fourth subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_LCA', hue='treatment_cam_only', data=df_cancer, marker='o', ax=axs[1, 1])

axs[1, 1].set_title('Patients with cancer')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Average Spending on Complementary Medicine')
axs[1, 1].legend(title='CAM - MHI usage', loc='upper left')
axs[1, 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.savefig(model_folder/'trends_treatment_cam_only_lca_cost.png', dpi = 300, bbox_inches = 'tight')


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# List of dataframes and titles
dataframes = [df_treated_filtered, df_healthy, df_multimorbid, df_cancer]
titles = ['All individuals', 'Individuals without chronic conditions', 'Multimorbid individuals', 'Individuals with cancer']

# Create subplots
for i, (ax, df, title) in enumerate(zip(axs.flat, dataframes, titles)):
    sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment', data=df, marker='o', ax=ax, legend=False)
    ax.set_title(title)
    ax.set_xlabel('Year')
    ax.set_ylabel('Conventional Medicine\nexpenditures (CHF)')
    ax.grid(True)

# Manually create legend handles and labels
legend_handles = [plt.Line2D([0], [0], marker='o', color=palette[i], 
                             label=dic_labels[treatment], markersize=8, linestyle='-') 
                  for i, treatment in enumerate(df_treated_filtered['treatment'].sort_values().unique())]

# Create a single legend with the correct title
legend = fig.legend(handles=legend_handles, 
           title='Complementary/Alternative Medicine (CAM)\nthrough Supplementary Insurance (SI)', 
           loc='lower center', bbox_to_anchor=(0.5, -0.04), ncol=2)
plt.setp(legend.get_title(), ha='center')  # 'ha' stands for horizontal alignment

# Adjust layout
plt.tight_layout()
fig.subplots_adjust(bottom=0.15)  # Make room for the legend

plt.savefig(model_folder/'trends_treatment_aos_cost.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# First subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment', data=df_treated_filtered, marker='o', ax=axs[0, 0])
axs[0, 0].set_title('All')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Expenditures CM - MHI (CHF)')
axs[0, 0].legend(title='CAM - SI usage', loc='upper left')
axs[0, 0].grid(True)
# Second subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment', data=df_healthy, marker='o', ax=axs[0, 1])
axs[0, 1].set_title('No-PCG individuals')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Expenditures CM - MHI (CHF)')
axs[0, 1].legend(title='CAM - SI usage', loc='upper left')
axs[0, 1].grid(True)

# Third subplot
# Create the line plot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment', data=df_multimorbid, marker='o', ax=axs[1, 0])
axs[1, 0].set_title('Multimorbid patients')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Expenditures CM - MHI (CHF)')
axs[1, 0].legend(title='CAM - SI usage', loc='upper left')
axs[1, 0].grid(True)

# Fourth subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment', data=df_cancer, marker='o', ax=axs[1, 1])

axs[1, 1].set_title('Patients with cancer')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Expenditures CM - MHI (CHF)')
axs[1, 1].legend(title='CAM - SI usage', loc='upper left')
axs[1, 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.savefig(model_folder/'trends_treatment_aos_cost.png', dpi = 300, bbox_inches = 'tight')


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# First subplot
sns.lineplot(x='NOANNEE', y='cds', hue='treatment', data=df_treated_filtered, marker='o', ax=axs[0, 0])
axs[0, 0].set_title('All')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Chronic disease score (CDS)')
axs[0, 0].legend(title='CAM - SI usage', loc='upper left')
axs[0, 0].grid(True)
# Second subplot
sns.lineplot(x='NOANNEE', y='cds', hue='treatment', data=df_healthy, marker='o', ax=axs[0, 1])
axs[0, 1].set_title('No-PCG individuals')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Chronic disease score (CDS)')
axs[0, 1].legend(title='Treatment', loc='upper left')
axs[0, 1].grid(True)

# Third subplot
# Create the line plot
sns.lineplot(x='NOANNEE', y='cds', hue='treatment', data=df_multimorbid, marker='o', ax=axs[1, 0])
axs[1, 0].set_title('Multimorbid patients')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Chronic disease score (CDS)')
axs[1, 0].legend(title='CAM - SI usage', loc='upper left')
axs[1, 0].grid(True)

# Fourth subplot
sns.lineplot(x='NOANNEE', y='cds', hue='treatment', data=df_cancer, marker='o', ax=axs[1, 1])

axs[1, 1].set_title('Patients with cancer')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Chronic disease score (CDS)')
axs[1, 1].legend(title='CAM - SI usage', loc='upper left')
axs[1, 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.savefig(model_folder/'trends_treatment_cds.png', dpi = 300, bbox_inches = 'tight')


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# List of dataframes and titles
dataframes = [df_treated_filtered, df_healthy, df_multimorbid, df_cancer]
titles = ['All', 'No-PCG individuals', 'Multimorbid patients', 'Patients with cancer']

# Create subplots
for i, (ax, df, title) in enumerate(zip(axs.flat, dataframes, titles)):
    sns.lineplot(x='NOANNEE', y='n_atc', hue='treatment', data=df, marker='o', ax=ax, legend=False)
    ax.set_title(title)
    ax.set_xlabel('Year')
    ax.set_ylabel('Number of distinct therapeutic subgroups')
    ax.grid(True)

# Manually create legend handles and labels
legend_handles = [plt.Line2D([0], [0], marker='o', color=palette[i], 
                             label=dic_labels[treatment], markersize=8, linestyle='-') 
                  for i, treatment in enumerate(df_treated_filtered['treatment'].unique())]

# Create a single legend with the correct title
fig.legend(handles=legend_handles, 
           title='Complementary/Alternative Medicine usage through Supplementary Insurance', 
           loc='lower center', bbox_to_anchor=(0.5, -0.04), ncol=2)
# Adjust layout
plt.tight_layout()
fig.subplots_adjust(bottom=0.15)  # Make room for the legend

plt.savefig(model_folder/'trends_treatment_atcs.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 6))
# First subplot
sns.lineplot(x='NOANNEE', y='n_atc', hue='treatment_cam_only', data=df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_AOS > df_treated_filtered.MTFRANCHISECOUV], marker='o', ax=axs[0, 0])
axs[0, 0].set_title('All')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Number of ATCs')
axs[0, 0].legend(title='CAM - MHI usage', loc='upper left')
axs[0, 0].grid(True)
# Second subplot
sns.lineplot(x='NOANNEE', y='n_atc', hue='treatment_cam_only', data=df_healthy, marker='o', ax=axs[0, 1])
axs[0, 1].set_title('No-PCG individuals')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Number of ATCs')
axs[0, 1].legend(title='CAM - MHI usage', loc='upper left')
axs[0, 1].grid(True)

# Third subplot
# Create the line plot
sns.lineplot(x='NOANNEE', y='n_atc', hue='treatment_cam_only', data=df_multimorbid, marker='o', ax=axs[1, 0])
axs[1, 0].set_title('Multimorbid patients')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Number of ATCs')
axs[1, 0].legend(title='CAM - MHI usage', loc='upper left')
axs[1, 0].grid(True)

# Fourth subplot
sns.lineplot(x='NOANNEE', y='n_atc', hue='treatment_cam_only', data=df_cancer, marker='o', ax=axs[1, 1])

axs[1, 1].set_title('Patients with cancer')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Number of ATCs')
axs[1, 1].legend(title='CAM - MHI usage', loc='upper left')
axs[1, 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.savefig(model_folder/'trends_treatment_atcs_MHI.png', dpi = 300, bbox_inches = 'tight')


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# First subplot
sns.lineplot(x='year', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cumulated', data=df_treated_filtered, marker='o', ax=axs[0, 0])
axs[0, 0].set_title('All')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Average Number of ATCs')
axs[0, 0].legend(title='Treatment', loc='upper left')
axs[0, 0].grid(True)
# Second subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cumulated', data=df_healthy, marker='o', ax=axs[0, 1])
axs[0, 1].set_title('No-PCG individuals')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Average Number of ATCs')
axs[0, 1].legend(title='Treatment', loc='upper left')
axs[0, 1].grid(True)

# Third subplot
# Create the line plot
sns.lineplot(x='year', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cumulated', data=df_multimorbid, marker='o', ax=axs[1, 0])
axs[1, 0].set_title('Multimorbid patients')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Average Number of ATCs')
axs[1, 0].legend(title='Treatment', loc='upper left')
axs[1, 0].grid(True)

# Fourth subplot
sns.lineplot(x='year', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cumulated', data=df_cancer, marker='o', ax=axs[1, 1])

axs[1, 1].set_title('Patients with cancer')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Average Number of ATCs')
axs[1, 1].legend(title='Treatment', loc='upper left')
axs[1, 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.savefig(model_folder/'trends_treatment_cumulated_aos_cost.png', dpi = 300, bbox_inches = 'tight')

## Cancer patients

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 4))

g = sns.barplot(data=df_treated_filtered, y="treatment", x='Cancer_PCG', width=.6, palette="vlag", ax=axs[0])
g.set_ylabel('Prevalence of CAM use (SI)')

g = sns.barplot(data=df_treated_filtered, y="treatment_cam_only", x='Cancer_PCG', width=.6, palette="vlag", ax=axs[1])
g.set_ylabel('Prevalence of CAM use (MHI)')

g = sns.barplot(data=df_treated_filtered, y="treatment_lca_cam", x='Cancer_PCG', width=.6, palette="vlag", ax=axs[2])
g.set_ylabel('Prevalence of CAM use (SI+MHI)')

plt.savefig(results_folder/'Prevalence_cancer_patients.png', dpi = 300, bbox_inches='tight')

In [ ]:
# Melt the dataframe to long format for methods columns.
methods_regex = 'Methodes.*_amount$'
methods_long_df = df_treated_filtered.melt(id_vars='Cancer_PCG', 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Method', value_name='Amount')


In [ ]:
methods_long_df['Method'] = methods_long_df['Method'].str.replace('_amount','')

In [ ]:
# Now use Seaborn to create a bar plot with error bars.
plt.figure(figsize=(14, 7))
sns.barplot(data=methods_long_df, x='Method', y='Amount', hue='Cancer_PCG', errorbar='ci')

# Improve the aesthetics and readability.
# plt.title('Comparison of Method Amounts by Cancer Classification with Error Bars')
plt.xlabel('Methods')
plt.ylabel('Average yearly CAM claims amount (SI) (CHF)')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability.
# plt.legend(title='Cancer_PCG')
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels.
plt.savefig(results_folder/'Costs_by_lca_method_cancer_patients.png', dpi = 300, bbox_inches='tight')

In [ ]:
for col in df_treated_filtered.filter(regex=methods_regex).columns:
    df_treated_filtered[f'{col}_binary'] = df_treated_filtered[col].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
methods_regex = 'Methodes.*_binary$'
methods_long_df = df_treated_filtered.melt(id_vars='Pain_PCG', 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Method', value_name='Prevalence')
methods_long_df['Method'] = methods_long_df['Method'].str.replace('amount_','')

In [ ]:
# Now use Seaborn to create a bar plot with error bars.
plt.figure(figsize=(14, 7))
sns.barplot(data=methods_long_df, x='Method', y='Prevalence', hue='Pain_PCG', errorbar='ci')

# Improve the aesthetics and readability.
# plt.title('Comparison of Method Amounts by Cancer Classification with Error Bars')
plt.xlabel('Methods')
plt.ylabel('Prevalence')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability.
# plt.legend(title='Cancer_PCG')
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels.
plt.savefig(results_folder/'Prevalence_by_lca_method_pain_patients.png', dpi = 300, bbox_inches='tight')

In [ ]:
methods_regex = 'Methodes.*_binary$'
methods_long_df = df_treated_filtered.melt(id_vars='Cancer_PCG', 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Method', value_name='Prevalence')
methods_long_df['Method'] = methods_long_df['Method'].str.replace('_amount_binary','')

In [ ]:
# Now use Seaborn to create a bar plot with error bars.
plt.figure(figsize=(14, 7))
sns.barplot(data=methods_long_df, x='Method', y='Prevalence', hue='Cancer_PCG', errorbar='ci')

# Improve the aesthetics and readability.
# plt.title('Comparison of Method Amounts by Cancer Classification with Error Bars')
plt.xlabel('Methods')
plt.ylabel('Prevalence')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability.
# plt.legend(title='Cancer_PCG')
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels.
plt.savefig(results_folder/'Prevalence_by_lca_method_cancer_patients.png', dpi = 300, bbox_inches='tight')

### Prevalence utilisation des CAM par PCG

In [ ]:
methods_regex = 'PCG'
methods_long_df = df_treated_filtered.melt(id_vars='treatment_cam_only', 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Disease', value_name='Prevalence')
methods_long_df['Disease'] = methods_long_df['Disease'].str.replace('_PCG','')

methods_long_df = methods_long_df.rename(columns = {"treatment_cam_only":'Treatment 3 (MHI only)'})

plt.figure(figsize=(14, 7))
sns.barplot(data=methods_long_df, x='Disease', y='Prevalence', hue='Treatment 3 (MHI only)', errorbar='ci')

# Improve the aesthetics and readability.
# plt.title('Comparison of Method Amounts by Cancer Classification with Error Bars')
plt.xlabel('Methods')
plt.ylabel('Prevalence')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability.
# plt.legend(title='Cancer_PCG')
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels.
plt.savefig(results_folder/'Prevalence_cam_only_by_pcg.png', dpi = 300, bbox_inches='tight')

In [ ]:
methods_regex = 'PCG'
methods_long_df = df_treated_filtered.melt(id_vars='treatment', 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Disease', value_name='Prevalence')
methods_long_df['Disease'] = methods_long_df['Disease'].str.replace('_PCG','')

methods_long_df = methods_long_df.rename(columns = {"treatment":'Treatment 1 (SI only)'})

plt.figure(figsize=(14, 7))
sns.barplot(data=methods_long_df, x='Disease', y='Prevalence', hue='Treatment 1 (SI only)', errorbar='ci')

# Improve the aesthetics and readability.
# plt.title('Comparison of Method Amounts by Cancer Classification with Error Bars')
plt.xlabel('Methods')
plt.ylabel('Prevalence')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability.
# plt.legend(title='Cancer_PCG')
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels.
plt.savefig(results_folder/'Prevalence_lca_only_by_pcg.png', dpi = 300, bbox_inches='tight')

## Heatmaps + HC - PCG vs methods (Figure 4)
### Heatmap SI

In [ ]:
methods_regex = 'Methodes.*_binary$'
# methods_long_df = df_treated_filtered.melt(id_vars='Cancer_PCG', 
#                           value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
#                           var_name='Method', value_name='Prevalence')
df_treated_filtered.filter(regex=methods_regex).columns

In [ ]:
methods_regex = 'PCG'
methods_long_df = df_treated_filtered.melt(id_vars=['uuid','year'], 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Disease', value_name='Prevalence')
methods_long_df['Disease'] = methods_long_df['Disease'].str.replace('_PCG','')

In [ ]:
methods_long_df = methods_long_df[methods_long_df.Prevalence == 1]

df_test = df_treated_filtered.filter(regex='Methodes.*binary$|uuid|year')

df_disease_methods = pd.merge(methods_long_df,df_test, on = ['uuid','year'])

In [ ]:
df_disease_methods[df_disease_methods['Disease'] == 'Asthma']['Methodes ayurvediques_amount_binary'].sum()

In [ ]:
df_disease_methods[df_disease_methods['Disease'] == 'Cancer']['Methodes ayurvediques_amount_binary'].sum()

In [ ]:
df_disease_methods = df_disease_methods.groupby('Disease').sum()

In [ ]:
exclude_columns = ['year','uuid','Prevalence']

for col in df_disease_methods.columns:
    if col not in exclude_columns:
        print(col)
        df_disease_methods[col] = df_disease_methods[col] / df_disease_methods['Prevalence']

In [ ]:
df_disease_methods = df_disease_methods.drop(['year','Prevalence'], axis = 1)

df_disease_methods.columns = df_disease_methods.columns.str.replace('_amount_binary','')

In [ ]:
method_mapping = {
    'Methodes ayurvediques': 'Ayurvedic methods',
    "Methodes d'art therapie": 'Art therapy methods',
    'Methodes de massage': 'Massage methods',
    'Methodes energetiques': 'Energetic methods',
    'Methodes energetiques manuelles': 'Manual energetic methods',
    'Methodes hydrotherapeutiques': 'Hydrotherapeutic methods',
    'Methodes occidentales': 'Western methods',
    'Methodes orientales': 'Oriental methods',
    'Methodes prescriptives': 'Prescriptive methods',
    'Methodes psychologiques complementaires': 'Complementary psychological methods',
    'Methodes reflexes': 'Reflex methods',
    'Methodes therapeutiques par le mouvement': 'Movement-based therapeutic methods'
}

disease_mapping = {

    'Asthma' : "Asthma",
  'Diabetes' : "Diabetes",
  'Cancer'   : "Cancer",
  'Epilepsy' : "Epilepsy",
  'Glaucoma' : "Glaucoma",
  'HIV_AIDS' : "HIV/AIDS",
  'Heart_disease' : "Heart Disease",
  'Hypertension_related' : "Hypertension",
  'Immune' : "Immune Disorders",
  'Inflammatory' : "Inflammatory Disorders",
  'Mental' : "Mental Health Conditions",
  'Other' : "Other Conditions",
  'Pain' : "Pain Related Conditions",
  'Parkinson' : "Parkinson's Disease",
  'Thyroid' : "Thyroid Disorders",
}

In [ ]:
df_disease_methods = df_disease_methods.rename(columns=method_mapping)
df_disease_methods = df_disease_methods.rename(index=disease_mapping)

In [ ]:
# Better color bar handling
cmap = sns.diverging_palette(220, 20, as_cmap=True)  # Create a custom diverging palette

# Generate the clustermap
g = sns.clustermap(
    df_disease_methods.drop('uuid',axis=1),
    z_score=1,
    cmap=cmap,
    figsize=(10, 10),  # Adjust size to fit labels if needed
    linewidths=.5,  # Add lines between cells to enhance readability
    cbar_kws={'label': 'Standardized Scores'}  # Label for the color bar
)
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0)  # Rotate the y labels for better readability
# plt.set_xlabel('CAM - SI methods')
# plt.setp(g.ax_heatmap.get_xticklabels(), rotation=45)  # Rotate the x labels for readability
# g.ax_heatmap.set_title('Typology based on Therapy Methods and PCGs', pad=-10)  # Set title with padding for visibility
plt.savefig(results_folder/'Heatmap_disease_SI_methods.png', dpi = 300, bbox_inches='tight')

### Heatmap MHI

In [ ]:
df_cdposition = pd.read_parquet(data_folder/'processed/Intermediate datasets/df_amount_by_cdposition.parquet.gzip')

In [ ]:
columns_to_update = ['Acupuncture', 'Anthroposophic medicine', 'Homeopathy', 'Neural therapy', 'Phytotherapy', 'Traditional Chinese medicine']
df_treated_filtered[[i+'_b' for i in columns_to_update]] = df_treated_filtered[columns_to_update].apply(lambda x: np.where(x > 0, 1, x))

In [ ]:
methods_regex = 'Acupuncture_b|Anthroposophic medicine_b|Homeopathy_b|Neural therapy_b|Phytotherapy_b|Traditional Chinese medicine_b'
# methods_long_df = df_treated_filtered.melt(id_vars='Cancer_PCG', 
#                           value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
#                           var_name='Method', value_name='Prevalence')
df_treated_filtered.filter(regex=methods_regex).columns

In [ ]:
methods_regex = 'PCG'
methods_long_df = df_treated_filtered.melt(id_vars=['uuid','year'], 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Disease', value_name='Prevalence')
methods_long_df['Disease'] = methods_long_df['Disease'].str.replace('_PCG','')

In [ ]:
methods_long_df = methods_long_df[methods_long_df.Prevalence == 1]

df_test = df_treated_filtered.filter(regex='uuid|year|Acupuncture_b|Anthroposophic medicine_b|Homeopathy_b|Neural therapy_b|Phytotherapy_b|Traditional Chinese medicine_b')
# g = sns.clustermap(df_test, method="average", cmap="coolwarm", standard_scale=1)

df_disease_methods = pd.merge(methods_long_df,df_test, on = ['uuid','year'])

In [ ]:
df_disease_methods = df_disease_methods.groupby('Disease').sum()

In [ ]:
exclude_columns = ['uuid','year','Prevalence']

for col in df_disease_methods.columns:
    if col not in exclude_columns:
        df_disease_methods[col] = df_disease_methods[col] / df_disease_methods['Prevalence']

In [ ]:
df_disease_methods = df_disease_methods.drop(['year','uuid','Prevalence'], axis = 1)
df_disease_methods.columns = df_disease_methods.columns.str.replace('_b','')

In [ ]:
# df_disease_methods = df_disease_methods.rename(columns=method_mapping)
df_disease_methods = df_disease_methods.rename(index=disease_mapping)

In [ ]:
# Generate the clustermap
cmap = sns.diverging_palette(220, 20, as_cmap=True)  # Create a custom diverging palette

g = sns.clustermap(
    df_disease_methods,
    z_score=1,
    cmap=cmap,
    figsize=(10, 10),  # Adjust size to fit labels if needed
    linewidths=.5,  # Add lines between cells to enhance readability
    cbar_kws={'label': 'Z-Scores'}  # Label for the color bar
)
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0) 
plt.savefig(results_folder/'Heatmap_disease_MHI_methods.png', dpi = 300, bbox_inches='tight')